#### Preparar Ambiente

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install tqdm

     |████████████████████████████████| 76 kB 5.6 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [5]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [6]:
from datetime import datetime, timedelta
import numpy as np

In [7]:
from tqdm.notebook import tqdm
from time import sleep

#### Crear tablas por pais

In [8]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [9]:
pais = 'BR'

#### Cuantil de distancia

In [10]:
tabla_historica_usuarios = f'historico_usuarios'
query = f'''
    SELECT iso_country_code, month, day, distancia_recorrida
    FROM {tabla_historica_usuarios}
    WHERE iso_country_code = '{pais}' and month = 02 and day = 05 '''
dfq = cursor.execute(query).as_pandas()
dfq.head()

,iso_country_code,month,day,distancia_recorrida
0,BR,2,5,2.392055
1,BR,2,5,0.000000
2,BR,2,5,7.486799
3,BR,2,5,52.348395
4,BR,2,5,0.211835


In [11]:
dfq.distancia_recorrida.quantile([0.25, 0.5, 0.75, 1.00])

0.25         0.000000
0.50         1.100152
0.75        16.277515
1.00    213076.179756
Name: distancia_recorrida, dtype: float64

#### Revisar tabla usuarios-pois

In [12]:
cat = 'banks'
#cat = 'all'

tabla_hogares_pois_pais = f'historico_hogares_pois_{cat}_{pais}'
query = f'''
    SELECT count(*) as n,
    COUNT(distinct (caid, iso_country_code)) as n_usuarios_unicos
    FROM {tabla_hogares_pois_pais}'''
cursor.execute(query).as_pandas()

,n,n_usuarios_unicos
0,460553,342306


In [13]:
historico_usuarios_pois_pais = f'historico_usuarios_pois_{cat}_{pais}'
query = f'''
    CREATE TABLE IF NOT EXISTS {historico_usuarios_pois_pais} AS
    SELECT  g.id_poi,
            h.caid,
            h.distancia_recorrida,
            h.lat_final,
            h.long_final,
            h.n_obs,
            h.n_obs_noche,
            h.n_obs_madrugada,
            h.day,
            h.month,
            h.year,
            h.iso_country_code
    FROM (SELECT caid,
                 id_poi
          FROM {tabla_hogares_pois_pais}) as g INNER JOIN (
            SELECT *
            FROM historico_usuarios
            WHERE iso_country_code='{pais}') as h
        ON h.caid=g.caid
'''
cursor.execute(query)

print(f'Generacion de tabla {historico_usuarios_pois_pais}: terminado')

Generacion de tabla historico_usuarios_pois_banks_BR: terminado


#### Cruzar tabla pois con tabla de pings por dia

In [ ]:
fechas = pd.date_range('09/01/2020','09/30/2020')

for fecha in fechas:
    mes = fecha.month
    dia = fecha.day
    codigo_mes = '0'+str(mes) if mes<10 else str(mes)
    codigo_dia = '0'+str(dia) if dia<10 else str(dia)
    codigo_fecha = codigo_mes + codigo_dia
    
    
    # Tabla historico pings con pois de hogar por dia
    #===========================================
    tabla_pings_poish_dia = f'pings_poish_{cat}_{pais}_{codigo_fecha}'
    query = f'''
        CREATE TABLE {tabla_pings_poish_dia} AS
        SELECT  g.id_poi,
                h.utc_timestamp,
                h.latitude,
                h.longitude,
                h.caid,
                h.id_type,
                h.hour_of_day,
                h.day,
                h.month,
                h.year,
                h.iso_country_code
        FROM (SELECT caid,
                     id_poi
              FROM {tabla_hogares_pois_pais}) as g INNER JOIN (
                SELECT *
                FROM historico_pings
                WHERE iso_country_code='{pais}' AND month={mes} AND day={dia}) as h
            ON h.caid=g.caid
    '''
    cursor.execute(query)
    time.sleep(20)
    #
    cursor.close()
    #print(f"Table {tabla_pings_poish_dia}: DONE ")
    
    
    # Asignando pois a tabla pings por dia
    #===========================================
    tabla_pings_poisp_dia = f'pings_poisp_{cat}_{pais}_{codigo_fecha}'
    query = f'''
                CREATE TABLE {tabla_pings_poisp_dia} AS
                SELECT utc_timestamp,
                    caid,
                    id_type,
                    hour_of_day,
                    day,
                    month,
                    year,
                    iso_country_code,
                    id as id_poip,
                    nom_gnc as nom_gncp,
                    banco as bancop,
                    cd_geocmi as cd_geocmip
                FROM {tabla_pings_poish_dia}
                CROSS JOIN pois_{cat}_{pais}
                WHERE ST_CONTAINS (ST_GeomFromLegacyBinary(pois_{cat}_{pais}.boundarypoi), ST_POINT({tabla_pings_poish_dia}.longitude, {tabla_pings_poish_dia}.latitude))
                ORDER BY  id'''
    cursor.execute(query)
    time.sleep(20)
    #
    cursor.close()
    #print(f"Table {tabla_pings_poisp_dia}: DONE ")
    
    
    # Asignando indicadores de usuarios a tabla de pings y pois por dia
    #===========================================
    tabla_pings_usuarios_fecha = f'pings_pois_{cat}_usuarios_{pais}_{codigo_fecha}'
    query = f'''
        CREATE TABLE {tabla_pings_usuarios_fecha} AS
        SELECT  {tabla_pings_poisp_dia}.id_poip,
                {tabla_pings_poisp_dia}.nom_gncp,
                {tabla_pings_poisp_dia}.bancop,
                {tabla_pings_poisp_dia}.cd_geocmip,
                {tabla_pings_poisp_dia}.utc_timestamp,
                {tabla_pings_poisp_dia}.caid,
                {tabla_pings_poisp_dia}.id_type,
                {tabla_pings_poisp_dia}.hour_of_day,
                {tabla_pings_poisp_dia}.day,
                {tabla_pings_poisp_dia}.month,
                {tabla_pings_poisp_dia}.year,
                {tabla_pings_poisp_dia}.iso_country_code,
                h.distancia_recorrida,
                h.lat_final,
                h.long_final,
                h.n_obs,
                h.n_obs_noche,
                h.n_obs_madrugada,
                h.id_poi as id_poih
        FROM {tabla_pings_poisp_dia} LEFT JOIN (SELECT * 
                                          FROM {historico_usuarios_pois_pais}
                                          WHERE month={mes} AND day={dia}) as h
            ON h.caid={tabla_pings_poisp_dia}.caid
            AND h.year={tabla_pings_poisp_dia}.year
            AND h.month={tabla_pings_poisp_dia}.month
            AND h.day={tabla_pings_poisp_dia}.day
    '''
    cursor.execute(query)
    #print(f"Table {tabla_pings_usuarios_fecha}: DONE ")
    
    print(f"FECHA {fecha} DONE ")

#### Crear tabla agrupada

In [16]:
fechas = pd.date_range('09/01/2020','09/30/2020')

for i,fecha in enumerate(fechas):
    mes = fecha.month
    dia = fecha.day
    codigo_mes = '0'+str(mes) if mes<10 else str(mes)
    codigo_dia = '0'+str(dia) if dia<10 else str(dia)
    codigo_fecha = codigo_mes + codigo_dia
    tabla_agrupada = f'historico_pois_{cat}_{pais}'
    tabla_pings_usuarios_fecha = f'pings_pois_{cat}_usuarios_{pais}_{codigo_fecha}'
    if i == 0 and (codigo_dia == '01' and codigo_mes == '02'):
        query_agrupar = f'''
        CREATE TABLE {tabla_agrupada} AS
        SELECT id_poi,
          nom_gnc,
          banco,
          cd_geocmi,
          month,
          day,
          hour_of_day,
          SUM(n_pings) AS n_pings,
          SUM(n_users) AS n_users,
          SUM(n_visitantes_poi) AS n_visitantes_poi,
          SUM(n_usuarios_hora) AS n_usuarios_hora,
          AVG(prom_dist) AS prom_dist,
          SUM(n_mas_de_1km) AS n_mas_de_1km,
          SUM(n_menos_de_1km) AS n_menos_de_1km,
          SUM(n_1a5_km) AS n_1a5_km,
          SUM(n_5a10_km) AS n_5a10_km,
          SUM(n_mas_de_10km) AS n_mas_de_10km,
          SUM(n_usuarios_android) AS n_usuarios_android,
          SUM(n_usuarios_ios) AS n_usuarios_ios
        FROM (
          SELECT id_poip as id_poi,
          nom_gncp as nom_gnc,
          bancop as banco,
          cd_geocmip as cd_geocmi,
          month,
          day,
          hour_of_day,
          COUNT(*) AS n_pings,
          COUNT(DISTINCT caid) AS n_users,
          COUNT(DISTINCT (caid, hour_of_day)) AS n_usuarios_hora,
          MAX(CASE WHEN id_poih<>id_poip then 1 ELSE 0 END) AS n_visitantes_poi,
          AVG(distancia_recorrida) AS prom_dist,
          MAX(CASE WHEN distancia_recorrida >= 1 then 1 ELSE 0 END) AS n_mas_de_1km,
          MAX(CASE WHEN distancia_recorrida < 1 then 1 ELSE 0 END) AS n_menos_de_1km,
          MAX(CASE WHEN (distancia_recorrida >= 1 AND distancia_recorrida <5) then 1 ELSE 0 END) AS n_1a5_km,
          MAX(CASE WHEN (distancia_recorrida >= 5 AND distancia_recorrida <10) then 1 ELSE 0 END) AS n_5a10_km,
          MAX(CASE WHEN distancia_recorrida >= 10 then 1 ELSE 0 END) AS n_mas_de_10km,
          MAX(CASE WHEN id_type='ANDROID' then 1 ELSE 0 END) AS n_usuarios_android,
          MAX(CASE WHEN id_type='IOS' then 1 ELSE 0 END) AS n_usuarios_ios
          FROM {tabla_pings_usuarios_fecha}
          GROUP BY id_poip,
          nom_gncp,
          bancop,
          cd_geocmip,
          month, 
          day,
          hour_of_day,
          caid
        )
        GROUP BY id_poi,
          nom_gnc,
          banco,
          cd_geocmi,
          month, 
          day,
          hour_of_day
        '''

        cursor.execute(query_agrupar)
    else:
        query_agrupar = f'''
        INSERT INTO {tabla_agrupada} 
        SELECT id_poi,
          nom_gnc,
          banco,
          cd_geocmi,
          month,
          day,
          hour_of_day,
          SUM(n_pings) AS n_pings,
          SUM(n_users) AS n_users,
          SUM(n_visitantes_poi) AS n_visitantes_poi,
          SUM(n_usuarios_hora) AS n_usuarios_hora,
          AVG(prom_dist) AS prom_dist,
          SUM(n_mas_de_1km) AS n_mas_de_1km,
          SUM(n_menos_de_1km) AS n_menos_de_1km,
          SUM(n_1a5_km) AS n_1a5_km,
          SUM(n_5a10_km) AS n_5a10_km,
          SUM(n_mas_de_10km) AS n_mas_de_10km,
          SUM(n_usuarios_android) AS n_usuarios_android,
          SUM(n_usuarios_ios) AS n_usuarios_ios
        FROM (
          SELECT id_poip as id_poi,
          nom_gncp as nom_gnc,
          bancop as banco,
          cd_geocmip as cd_geocmi,
          month,
          day,
          hour_of_day,
          COUNT(*) AS n_pings,
          COUNT(DISTINCT caid) AS n_users,
          COUNT(DISTINCT (caid, hour_of_day)) AS n_usuarios_hora,
          MAX(CASE WHEN id_poih<>id_poip then 1 ELSE 0 END) AS n_visitantes_poi,
          AVG(distancia_recorrida) AS prom_dist,
          MAX(CASE WHEN distancia_recorrida >= 1 then 1 ELSE 0 END) AS n_mas_de_1km,
          MAX(CASE WHEN distancia_recorrida < 1 then 1 ELSE 0 END) AS n_menos_de_1km,
          MAX(CASE WHEN (distancia_recorrida >= 1 AND distancia_recorrida <5) then 1 ELSE 0 END) AS n_1a5_km,
          MAX(CASE WHEN (distancia_recorrida >= 5 AND distancia_recorrida <10) then 1 ELSE 0 END) AS n_5a10_km,
          MAX(CASE WHEN distancia_recorrida >= 10 then 1 ELSE 0 END) AS n_mas_de_10km,
          MAX(CASE WHEN id_type='ANDROID' then 1 ELSE 0 END) AS n_usuarios_android,
          MAX(CASE WHEN id_type='IOS' then 1 ELSE 0 END) AS n_usuarios_ios
          FROM {tabla_pings_usuarios_fecha}
          GROUP BY id_poip,
          nom_gncp,
          bancop,
          cd_geocmip,
          month, 
          day,
          hour_of_day,
          caid
        )
        GROUP BY id_poi,
          nom_gnc,
          banco,
          cd_geocmi,
          month, 
          day,
          hour_of_day
        '''

        cursor.execute(query_agrupar)  
        time.sleep(20)
        
    print(f"FECHA {fecha} DONE ")

FECHA 2020-09-01 00:00:00 DONE 
FECHA 2020-09-02 00:00:00 DONE 
FECHA 2020-09-03 00:00:00 DONE 
FECHA 2020-09-04 00:00:00 DONE 
FECHA 2020-09-05 00:00:00 DONE 
FECHA 2020-09-06 00:00:00 DONE 
FECHA 2020-09-07 00:00:00 DONE 
FECHA 2020-09-08 00:00:00 DONE 
FECHA 2020-09-09 00:00:00 DONE 
FECHA 2020-09-10 00:00:00 DONE 
FECHA 2020-09-11 00:00:00 DONE 
FECHA 2020-09-12 00:00:00 DONE 
FECHA 2020-09-13 00:00:00 DONE 
FECHA 2020-09-14 00:00:00 DONE 
FECHA 2020-09-15 00:00:00 DONE 
FECHA 2020-09-16 00:00:00 DONE 
FECHA 2020-09-17 00:00:00 DONE 
FECHA 2020-09-18 00:00:00 DONE 
FECHA 2020-09-19 00:00:00 DONE 
FECHA 2020-09-20 00:00:00 DONE 
FECHA 2020-09-21 00:00:00 DONE 
FECHA 2020-09-22 00:00:00 DONE 
FECHA 2020-09-23 00:00:00 DONE 
FECHA 2020-09-24 00:00:00 DONE 
FECHA 2020-09-25 00:00:00 DONE 
FECHA 2020-09-26 00:00:00 DONE 
FECHA 2020-09-27 00:00:00 DONE 
FECHA 2020-09-28 00:00:00 DONE 
FECHA 2020-09-29 00:00:00 DONE 
FECHA 2020-09-30 00:00:00 DONE 
